In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer
import lightgbm as lgb
import xgboost as xgb
import warnings
from tqdm.auto import tqdm

warnings.filterwarnings('ignore')

# 1. 텍스트 임베딩 함수 (이전과 동일)
def get_embeddings(data, model, tokenizer):
    embeddings = []
    for text in tqdm(data, desc="텍스트 임베딩 진행 중"):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=50)
        with torch.no_grad():
            outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()
        embeddings.append(cls_embedding)
    return np.vstack(embeddings)

# ⭐️ 2. 우선순위를 적용한 브랜드 추출 함수 및 리스트 정의
def extract_brand(apt_name, brand_list):
    """아파트 이름에서 우선순위가 가장 높은 브랜드명을 추출하는 함수"""
    for brand in brand_list:
        if brand in apt_name:
            return brand  # 리스트의 앞 순서(더 높은 우선순위)에 있는 브랜드를 먼저 반환
    return '기타 브랜드'

# 국내 주요 아파트 브랜드 '우선순위' 리스트
brand_priority_list = [
    # --- 최상위 프리미엄 브랜드 ---
    '디에이치', '아크로', '르엘', '오티에르',
    # --- 1군 메이저 브랜드 ---
    '자이', '힐스테이트', '래미안', '푸르지오', '더샵', '롯데캐슬', '아이파크', 'e편한세상', 'SK뷰', '포레나',
    # --- 중견/강소 브랜드 ---
    '어울림', '데시앙', '호반써밋', '중흥S-클래스', '우미린', '서희스타힐스', '한라비발디',
    '하늘채', '베르디움', '스위첸', '꿈에그린', '반도유보라', '제일풍경채', '금강펜테리움',
    '동문굿모닝힐', '신동아파밀리에', '코아루', '두산위브', '쌍용예가', '이수브라운스톤',
    '태영데시앙', '계룡리슈빌', '화성파크드림', '우방아이유쉘', '삼정그린코아', '일성트루엘',
    '대방노블랜드', '대광로제비앙', '양우내안애', '경남아너스빌', '삼부르네상스', '한양수자인',
    '신안인스빌', '파라곤', '골드클래스', '시티프라디움', '해링턴플레이스', '모아엘가',
    '빌리브', '지웰', '트루엘', '칸타빌',
    # --- 서브 브랜드 및 단지명 특징 (가장 낮은 우선순위) ---
    '더퍼스트', '더원', '더리브', '리더스포레', '트리우스', '그라시엘',
    '리버파크', '센트럴파크', '디오션', '에듀포레', '파크뷰', '레이크'
]


# 3. 데이터 로드 및 특성 공학
df = pd.read_csv("final_data.csv")
df['특별분양'] = pd.to_numeric(df['특별분양'], errors='coerce').fillna(0)
df['기준년월'] = pd.to_datetime(df['기준년월'])
df['년'] = df['기준년월'].dt.year
df['월'] = df['기준년월'].dt.month
df['일반분양'] = df['일반분양'].replace('-', 0).astype('int64')
df['특별분양'] = df['특별분양'].replace('-', 0).astype('int64')
df['일반분양'] = df['일반분양'].astype('int64')
df['특별분양'] = df['특별분양'].astype('int64')
df.drop(columns=['기준년월', '미분양수', '주변시세 평균'], inplace=True, errors='ignore')
for col in ['분양가(만원)', '공급면적(㎡)', '주변시세 평균(만원)', '세대수']:
    if col in df.columns:
        df[col].replace(0, 1, inplace=True)
infra_cols = [col for col in df.columns if 'km' in col or '500m' in col]
df['평당분양가'] = df['분양가(만원)'] / (df['공급면적(㎡)'] / 3.3)
df['인프라_점수'] = df[infra_cols].sum(axis=1)
df['시세초과비율'] = ((df['분양가(만원)'] - df['주변시세 평균(만원)']) / df['주변시세 평균(만원)']).replace([np.inf, -np.inf], np.nan)
df['시세차익률'] = (df['시세차익(만원)'] / df['분양가(만원)']).replace([np.inf, -np.inf], np.nan)
df['전용률'] = (df['전용면적(㎡)'] / df['공급면적(㎡)']) * 100
df['세대당면적'] = df['공급면적(㎡)'] / df['세대수']
df['특별분양유무'] = df['특별분양'].apply(lambda x: 1 if x > 0 else 0)
if '금리' in df.columns:
    df['금리구간'] = pd.cut(df['금리'], bins=[1, 2.5, 3.0, 3.5, np.inf],
                            labels=['1~2.5%', '2.5~3.0%', '3.0~3.5%', '3.5%~'], right=False)

# 4. 브랜드 추출 및 임베딩 생성
print("🏙️ 아파트 이름에서 브랜드를 추출합니다...")
df['브랜드'] = df['아파트'].apply(lambda x: extract_brand(str(x), brand_priority_list))
print("--- 브랜드 추출 결과 (상위 5개) ---")
print(df[['아파트', '브랜드']].head())

MODEL_NAME = "kykim/bert-kor-base"
print(f"\n🤖 '{MODEL_NAME}' 모델과 토크나이저를 로드합니다...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

brand_embeddings = get_embeddings(df['브랜드'], model, tokenizer)
co_embeddings = get_embeddings(df['건설사'], model, tokenizer)

brand_embed_df = pd.DataFrame(brand_embeddings, columns=[f'brand_embed_{i}' for i in range(brand_embeddings.shape[1])])
co_embed_df = pd.DataFrame(co_embeddings, columns=[f'co_embed_{i}' for i in range(co_embeddings.shape[1])])

df_processed = pd.concat([df.reset_index(drop=True), brand_embed_df, co_embed_df], axis=1)

# 5. 모델링 준비
target = '분양률'
drop_cols = infra_cols + ['아파트', '브랜드', '건설사', '주변시세 평균(만원)', '시세차익(만원)', '분양률']
X = df_processed.drop(columns=drop_cols, errors='ignore')
y = df_processed[target]
y.fillna(y.mean(), inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

# 6. 전처리 및 모델 훈련
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())]), numerical_features),
        ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')), ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), categorical_features)
    ],
    remainder='passthrough'
)
print("\n데이터 전처리를 시작합니다...")
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)
print("✅ 데이터 전처리 완료")

lgbm = lgb.LGBMRegressor(random_state=42, verbose=-1)
xgb_model = xgb.XGBRegressor(random_state=42)
rf = RandomForestRegressor(random_state=42)
param_dist_lgbm = {'n_estimators': [300, 500], 'learning_rate': [0.05, 0.1], 'max_depth': [5, 7], 'num_leaves': [31, 63]}
param_dist_xgb = {'n_estimators': [300, 500], 'learning_rate': [0.05, 0.1], 'max_depth': [5, 7], 'subsample': [0.8, 0.9]}
param_dist_rf = {'n_estimators': [300, 500], 'max_depth': [10, None], 'min_samples_split': [2, 5]}
print("\n🚀 모델 최적화(RandomizedSearchCV)를 시작합니다...")
search_lgbm = RandomizedSearchCV(lgbm, param_dist_lgbm, n_iter=10, cv=3, n_jobs=-1, random_state=42, scoring='r2')
search_xgb = RandomizedSearchCV(xgb_model, param_dist_xgb, n_iter=10, cv=3, n_jobs=-1, random_state=42, scoring='r2')
search_rf = RandomizedSearchCV(rf, param_dist_rf, n_iter=10, cv=3, n_jobs=-1, random_state=42, scoring='r2')
search_lgbm.fit(X_train_processed, y_train)
print("✅ LightGBM 최적화 완료")
search_xgb.fit(X_train_processed, y_train)
print("✅ XGBoost 최적화 완료")
search_rf.fit(X_train_processed, y_train)
print("✅ RandomForest 최적화 완료")
best_lgbm = search_lgbm.best_estimator_
best_xgb = search_xgb.best_estimator_
best_rf = search_rf.best_estimator_

print("\n🔥 스태킹 모델 훈련을 시작합니다...")
stack_model = StackingRegressor(
    estimators=[('lgbm', best_lgbm), ('xgb', best_xgb), ('rf', best_rf)],
    final_estimator=GradientBoostingRegressor(n_estimators=100, learning_rate=0.05, random_state=42),
    cv=3,
    passthrough=True,
    n_jobs=-1
)
stack_model.fit(X_train_processed, y_train)

y_pred = stack_model.predict(X_test_processed)
print("\n🏁 최종 성능 결과 (브랜드 우선순위 임베딩 적용):")
print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"R² Score: {r2_score(y_test, y_pred):.4f}")

🏙️ 아파트 이름에서 브랜드를 추출합니다...
--- 브랜드 추출 결과 (상위 5개) ---
         아파트 브랜드
0  강릉자이르네디오션  자이
1  강릉자이르네디오션  자이
2  강릉자이르네디오션  자이
3  강릉자이르네디오션  자이
4  강릉자이르네디오션  자이

🤖 'kykim/bert-kor-base' 모델과 토크나이저를 로드합니다...


tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/476M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/476M [00:00<?, ?B/s]

텍스트 임베딩 진행 중:   0%|          | 0/2214 [00:00<?, ?it/s]

텍스트 임베딩 진행 중:   0%|          | 0/2214 [00:00<?, ?it/s]


데이터 전처리를 시작합니다...
✅ 데이터 전처리 완료

🚀 모델 최적화(RandomizedSearchCV)를 시작합니다...
✅ LightGBM 최적화 완료
✅ XGBoost 최적화 완료
✅ RandomForest 최적화 완료

🔥 스태킹 모델 훈련을 시작합니다...

🏁 최종 성능 결과 (브랜드 우선순위 임베딩 적용):
MSE: 0.0440
MAE: 0.1406
R² Score: 0.5325
